# <center>Crash course 1: Vectors and matrices in numpy and scipy</center>
### <center>Alfred Galichon (NYU & ScPo) and Clément Montes (ScPo)</center>
## <center>'math+econ+code' masterclass on optimal transport and economic applications</center>
#### <center>With python code examples</center>
© 2018-2022 by Alfred Galichon. Past and present support from NSF grant DMS-1716489, ERC grant CoG-866274 are acknowledged, as well as inputs from contributors listed [here](http://www.math-econ-code.org/theteam).

**If you reuse material from this masterclass, please cite as:**<br>
Alfred Galichon, 'math+econ+code' masterclass on optimal transport and economic applications, January 2022. https://github.com/math-econ-code/mec_optim

# Introducing NumPy

* Unlike R or Matlab, Python has no built-in matrix algebra interface. Fortunately, the NumPy library provides powerful matrix capabilities, on par with R or Matlab. Here is a quick introduction to vectorization, operations on vectors and matrices, higher-dimensional arrays, Kronecker products and sparse matrices, etc. in NumPy.

* This is *not* a tutorial on Python itself. They are plenty good ones available on the web.

* First, we load numpy (with its widely used alias):

In [ ]:
import numpy as np

In NumPy, an `array` is built from a lists as follows:

In [ ]:
u = np.array([1,2,3])
print(u)
v = np.array([3,2,5])
print(v)

One can then add arrays as:

In [ ]:
print(np.array([1,2,3])+np.array([3,2,5]))

Note the difference between the + operator when applied to numpy arrays vs. when applied to lists:

In [ ]:
[1,2,3]+[3,2,5]

In the latter case, it returns list concatenation.

To input matrices in NumPy, one simply inputs a list of rows, which are themselves represented as lists.

In [ ]:
A = np.array([[11,12],[21,22],[31,32]])
A

The `shape` attribute of an array indicated the dimension of that array.

In [ ]:
A.shape

## Vectorization and memory order

* Matrices in all mathematical softwares are represented in a *vectorized* way as a sequence of numbers in the computers memory. This representation can involve either stacking the lines, or stacking the columns.

* Different programming languages can use either of the two stacking conventions:
    + Stacking the lines (Row-major order) is used by `C`, and is the default convention for Python (NumPy). A matrix $M$ is represented by varying the last index first, i.e. a $2\times2$ matrix will be represented as $vec_C\left(M\right) = \left(M_{11}, M_{12}, M_{21}, M_{22}\right).$ 
    + Stacking the columns (Column-major order) is used by `Fortran`, `Matlab`, `R`, and most underlying core linear algebra libraries (like BLAS). A 2x2x2 3-dimensional array $A$ will be represented by varying the first index first, then the second, i.e. $vec_C\left(A\right) = \left( A_{111}, A_{112}, A_{121}, A_{122}, A_{211}, A_{212}, A_{221}, A_{222} \right)$. 

The command `flatten()` provides the vectorized representation of a matrix.

In [ ]:
A.flatten()

Remember, NumPy represents matrices by **varying the last index first**.

In order to reshape the matrix `a`, one modifies its `shape` attribute. The following reshapes the matrix `a` into a row vector. 

In [ ]:
A.shape = 1,6
A

The previous output evidences the fact that Python uses the row-major order: rows are stacked one after the other. 
To reshape the vector into a column vector, do:

In [ ]:
A.shape = 6,1
A

Equivalently, one could have set `A.shape=6,-1`, where Python would replace `-1` by the integer needed for the formula to make sense (in this case, `1`). 
Another way to reshape is to use the method `reshape,` which returns a duplicate of the object with the requested shape.

In [ ]:
A1=np.array(range(6))
A2 = A1.reshape(3,2)
print("A1=\n", A1)
print("A2=\n",A2)

Note that `NumPy` also supports the column-major order, but you have to specifically ask for it, by passing the optional argument `order='F'`, where 'F' stands for `Fortran`.

In [ ]:
A3 = np.array(range(6)).reshape(3,2, order='F')
A3

# Multiplication 

### Multiplication of arrays

There are several ways to multiply two arrays using NumPy. The most commonly used is the following.

In [ ]:
A = np.ones((2,2))
B = 3*np.eye(2)
A@B #@ is left associative. If you have A@B@C, it will compute (A@B)@C

Note that `np.matmul(A,B)` would give the same result as well, but it is more difficult to read `np.matmul(A,np.matmul(B,C))` than `A@B@C`.

### Multiplication by a scalar

In [ ]:
4*np.eye(2)

The above assignation of B corresponds to the multiplication by a scalar. It is the simplest broadcasting allowed by numpy (which makes this library more powerful than just using lists -it is also much quicker-). More on broadcasting will arrive later in that Notebook.

## Kronecker product

A very important identity is
\begin{align*}
vec_C\left(AXB\right) = \left(  A\otimes B^\top\right)  vec_C\left(X\right),
\end{align*}
where $vec_C$ is the vectorization under the C (row-major) order, and where the Kronecker product $\otimes$ is defined as follows for 2x2 matrices (with obvious generalization):

\begin{align*}
A\otimes B=
\begin{pmatrix}
a_{11}B & a_{12}B\\
a_{21}B & a_{22}B
\end{pmatrix}.
\end{align*}



In [ ]:
A = np.eye(2)

AXB = np.kron(A, B)
print("A=",A)
print("B=",B)
print("AXB=",AXB)

## Type broadcasting in NumPy


The term broadcasting describes how NumPy treats arrays with different shapes during arithmetic operations. 

Subject to certain constraints, the smaller array is “broadcasted” across the larger array so that they have compatible shapes. Broadcasting provides a means of vectorizing array operations.

In [ ]:
A = 10*np.array([[1],[2],[3]]) #Simplest broadcasting
B =  np.array([1,2])
print('A=\n',A)
print('B=\n',B)
print('A+B=\n',A+B)

The operation `A[:,np.newaxis]` creates a new dimension.

In [ ]:
v = np.array([3,4,5])
print(v)
print(v[:,np.newaxis])
print(v[np.newaxis,:])

# Arrays of larger dimensions

In [ ]:
a_3d_array = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
a_3d_array

Standard functions can also support arrays with more than 2 dimensions.

In [ ]:
a_multiarray = np.zeros((2,3,3,3))
print(a_multiarray, a_multiarray.shape)

# Searching for a maximum

### Maximum between 2 arrays

To compare two arrays (say $x$ and $y$) component by component, it is convenient to use `np.maximum`. It returns an array $z$ such that $ \forall i: z[i] = \max(x[i],y[i])$. 

In [ ]:
np.maximum(np.array([2, 3, 4]), np.array([1, 5, 2]))

You can even broadcast.

In [ ]:
np.maximum(np.eye(2), [0.5, 2]) # broadcasting

### Highest component within an array

`np.max` and `np.argmax` respectively find the maximum entry of a given array along a specified axis, and its index. `np.min` and `np.argmin` perform similar functions.

In [ ]:
A = np.array([[0, 1,3], [0, 5,7]])
A

In [ ]:
A.max(axis=0)

In [ ]:
A.argmax(axis=0)

In [ ]:
A.max(axis=1)

In [ ]:
A.min(axis=0)

In [ ]:
A.argmin(axis=0)

If `axis` is not specified, the maximum will be taken over all the entries of the matrix.

In [ ]:
np.max(A) 

Note: if your array contains a nan, you can use `np.nanmax` in order to ignore those values while searching for the highest component.

## Summing all elements of an array

In a similar fashion as above, `np.sum` sums the elements of an array over a given axis.

In [ ]:
A.sum( axis=0)

In [ ]:
A.sum(axis=1)

If `axis` is not specified, the sum is done over all the entries of the matrix.


In [ ]:
A.sum()

# Sparse matrices in Scipy

Sparse matrices are available in the `sparse` module of the `scipy` library. 

In [ ]:
import scipy.sparse as spr

In [ ]:
n = 1000

print('size of sparse identity matrix of size '+str(n) +' in MB = ' + str(spr.identity(n).data.size  / (1024**2)))

print('size of dense identity matrix of size '+str(n) +' in MB  = ' + str(spr.identity(n).todense().nbytes  / (1024**2)))

Working with sparse matrices requires less storage. It is explained by the fact that while a dense matrix needs to encode every coefficient on a byte, sparse matrices only store the non-null coefficients. It is really convenient to work with such objects when it comes to matrices with really high sizes.

In [ ]:
spr.identity(1000).data.size  , spr.identity(1000).todense().nbytes 

## Creating sparse matrices...

### ... with standard forms

In [ ]:
I5 = spr.identity(5)
I5

You can convert your sparse matrix into a dense one in order to visualise it. 

In [ ]:
I5.todense()

### ... from a dense matrix

Let's create a dense matrix and make it sparse.

In [ ]:
# import uniform module to create random numbers
from scipy.stats import uniform

In [ ]:
np.random.seed(seed=42)
dense_matrix = uniform.rvs(size=16, loc = 0, scale=2) #List of 16 random draws between 0 and 2
dense_matrix = np.reshape(dense_matrix, (4, 4))
dense_matrix

In [ ]:
dense_matrix[dense_matrix < 1] = 0 #Arbitrar criterion
dense_matrix

In [ ]:
sparse_matrix = spr.csr_matrix(dense_matrix)
print(sparse_matrix) #It prints a tuple giving the row and columns of the non-null component and its value.

### ... from scratch

You can create two arrays containing respectively the rows and the column of the non-null coefficients.
A third array would give the value of the non-null coefficient. The result is as follows:

In [ ]:
# row indices
row_ind = np.array([0, 1, 1, 3, 4])
# column indices
col_ind = np.array([0, 2, 4, 3, 4])
# coefficients
data = np.array([1, 2, 3, 4, 5], dtype=float)

mat_coo = spr.coo_matrix((data, (row_ind, col_ind)))
print(mat_coo)

### Every common operation seen below works with sparse matrices.

In [ ]:
I5 = spr.identity(5)
I5 + np.ones((5,5))

In [ ]:
I5 + np.diag([1.,2.,3.,4.,5.])

In [ ]:
I5 @ np.diag([1.,2.,3.,4.,5.])

In [ ]:
kron_product = spr.kron(I5 , 10 *np.array([[1,2],[3,4]]))

In [ ]:
kron_product.todense()